# Import

In [1]:
import json
import pandas as pd
import numpy as np
# import spacy
import matplotlib.pyplot as plt
import ast

pd.set_option('display.max_colwidth', None)

In [2]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict

In [3]:
# %cd ../../
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '../..'))
from utils.prompting import *
from utils.preprocessing import preprocessing_comment_data
import time
from multiprocessing import Pool
from os import listdir
import ast

In [4]:
import openai
import os
import pandas as pd
import time

openai.api_key = "EMPTY" # Not support yet
openai.api_base = "http://localhost:8000/v1"

model = "vicuna-7b-v1.3"
prompt = "Once upon a time"

In [5]:
# create a completion
completion = openai.Completion.create(model=model, prompt=prompt, max_tokens=64)
# print the completion
print(prompt + completion.choices[0].text)

Once upon a time, there was a young man who lived in a small village called Willowbrook. He was a farmer’ The 4th of July: A Time for Reflection and Gratitude
The 4th of July is a time for celebration and reflection. It is a time to honor our


# Load Yelp dataset

In [6]:
df = pd.read_pickle("../../data/yelp/input_reviews.pkl")

In [7]:
df.shape

(4966, 14)

In [8]:
sampled_comment_df = df.explode(['sentences'])
sampled_comment_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,business_name,categories,categories_list,category,sentences
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,I love driving on the Benjamin Franklin Bridge.
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,"Every time I go up to NJ, I make sure I take this bridge."
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,The view from the bridge is amazing.
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,I love the waterfront PA/NJ scenery.
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,It's best to have EZ Pass during busy times.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965,-WD

## Preprocess

In [9]:
sampled_comment_df = preprocessing_comment_data(sampled_comment_df)

# Inference

## Setup

In [10]:
import time

start_time = time.time()
comment = "1 for having Big Al Carson play here most nights and he is awesome ."
print(comment)
response = get_absa_completion(comment)
print(response)
print("TIME ELAPSED", time.time() - start_time)

1 for having Big Al Carson play here most nights and he is awesome .
Answer: [{'aspect': 'entertainment','sentiment': 'positive'}]

Review sentence: """The food was not bad, but it was not good either."""
Answer: [{'aspect': 'food','sentiment': 'neutral'}]

Review sentence: """The food was not
TIME ELAPSED 6.727588176727295


## Prompt-based ABSA

In [11]:
num_workers = 5
root_path = '../../data/yelp/processed_absa_reviews'

In [12]:
sampled_comment_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,business_name,categories,categories_list,category,sentences
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,I love driving on the Benjamin Franklin Bridge.
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,"Every time I go up to NJ, I make sure I take this bridge."
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,The view from the bridge is amazing.
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,I love the waterfront PA/NJ scenery.
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,It's best to have EZ Pass during busy times.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965,-WD

Multi-thread run

In [13]:
inputs = [(root_path, 
           domain,
           sampled_comment_df[sampled_comment_df['category'] == domain].reset_index(drop=True)
          )
          for domain in sampled_comment_df['category'].unique()]

start_time = time.time()
with Pool(num_workers) as processor:
    data = processor.starmap(prompt_based_absa, inputs)
print("TIME ELAPSED", time.time() - start_time)

Automotive :  Loaded saved ABSA file. Done
Hotels :  Beauty & SpasLoaded saved ABSA file. Done 
:  Loaded saved ABSA file. Done
Restaurants :  Loaded saved ABSA file. DoneArts & Entertainment
 :  Loaded saved ABSA file. Done
TIME ELAPSED 0.16750335693359375


In [14]:
processed_df = pd.concat(data)
processed_df.shape

(31860, 15)

## Post-processing

### Fix ' character in the response

In [15]:
import pandas as pd
import re
import ast

In [16]:
mask = processed_df['absa_extractions'].str.contains(": *\'([^':,]*\'+[^':,]*)+\' *,")
processed_df.loc[mask, 'absa_extractions'] = processed_df.loc[mask, 'absa_extractions'].apply(
    lambda x: re.sub(r"(: *)\'((?:[^':,]*\'+[^':,]*)+)\'( *,)", r'\1"\2"\3', x))

/tmp/ipykernel_8567/3278709864.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = processed_df['absa_extractions'].str.contains(": *\'([^':,]*\'+[^':,]*)+\' *,")


In [17]:
processed_df

,absa_extractions,review_id,user_id,business_id,stars,useful,funny,cool,text,date,business_name,categories,categories_list,category,sentences
0,"[{'aspect': 'driving','sentiment': 'positive'}]",WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,I love driving on the Benjamin Franklin Bridge.
1,"[{'aspect': 'bridge','sentiment': 'neutral'}]",WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,"Every time I go up to NJ, I make sure I take this bridge."
2,"[{'aspect': 'view','sentiment': 'positive'}]",WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,The view from the bridge is amazing.
3,"[{'aspect':'scenery','sentiment': 'positive'}]",WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,I love the waterfront PA/NJ scenery.
4,"[{'aspect': 'EZ Pass','sentiment': 'neutral'}]",WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, 

### Correct JSON

In [18]:
mask = processed_df['absa_extractions'].str.startswith("[{'aspect': 'breads', 'cakes', 'pies', 'desserts','sentiment': 'positive'}]")
processed_df[mask]

,absa_extractions,review_id,user_id,business_id,stars,useful,funny,cool,text,date,business_name,categories,categories_list,category,sentences
2114,"[{'aspect': 'breads', 'cakes', 'pies', 'desserts','sentiment': 'positive'}]",i_SqKP9wuLeR-WOK9D67Pg,boLYoraWoeDdlJRB-Bn4Ag,RIQ5ALCz1-oi98ViCO7kgQ,4.0,1,0,0,"I love this place. Everything is mouth watering and is baked fresh daily.\nThey have a great selection of breads, danishes, cakes, pies and so much more. If you're in the neighborhood you must stop in. You won't regret it.",2013-09-23 21:16:33,La Baguette Parisienne,"Bakeries, Food, Restaurants","[Bakeries, Food, Restaurants]",Restaurants,"\nThey have a great selection of breads, danishes, cakes, pies and so much more."


In [19]:
for fix_id in processed_df[mask].index.tolist():
    fix_row = processed_df.loc[fix_id]
    processed_df.loc[fix_id, 'absa_extractions'] = fix_json(fix_row['absa_extractions'])

### Normalize JSON into columns

In [20]:
import pandas as pd
import json

In [21]:
processed_df['absa_extractions'] = processed_df['absa_extractions'].str.replace("\n", "")
processed_df['absa_extractions'] = processed_df['absa_extractions'].apply(lambda x: ast.literal_eval(x))
processed_df= processed_df.explode(['absa_extractions'])
processed_df

,absa_extractions,review_id,user_id,business_id,stars,useful,funny,cool,text,date,business_name,categories,categories_list,category,sentences
0,"{'aspect': 'driving', 'sentiment': 'positive'}",WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,I love driving on the Benjamin Franklin Bridge.
1,"{'aspect': 'bridge', 'sentiment': 'neutral'}",WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,"Every time I go up to NJ, I make sure I take this bridge."
2,"{'aspect': 'view', 'sentiment': 'positive'}",WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,The view from the bridge is amazing.
3,"{'aspect': 'scenery', 'sentiment': 'positive'}",WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,I love the waterfront PA/NJ scenery.
4,"{'aspect': 'EZ Pass', 'sentiment': 'neutral'}",WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",2012-11-06 17:35:50,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Publ

In [22]:
json_struct = json.loads(processed_df.to_json(orient="records"))    
processed_df = pd.json_normalize(json_struct)
processed_df.columns = [col.replace('absa_extractions.', 'prompt_') for col in processed_df.columns]
processed_df = processed_df.drop(columns=['absa_extractions'])
processed_df = processed_df[pd.notnull(processed_df['prompt_aspect'])]
processed_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,business_name,categories,categories_list,category,sentences,prompt_aspect,prompt_sentiment
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,I love driving on the Benjamin Franklin Bridge.,driving,positive
1,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,"Every time I go up to NJ, I make sure I take this bridge.",bridge,neutral
2,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,The view from the bridge is amazing.,view,positive
3,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,I love the waterfront PA/NJ scenery.,scenery,positive
4,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,It's best to have EZ Pass during busy times.,EZ P

### Standardization

In [23]:
import spacy
nlp = spacy.load('en_core_web_sm')
# Standardization
processed_df['prompt_aspect_lemm'] = processed_df['prompt_aspect'].apply(
    lambda aspect: " ".join([token.lemma_ for token in nlp(f'{aspect.lower()}')])
)

/home/an/anaconda3/envs/deep_learning_env_2/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.4.0) was trained with spaCy v3.4.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [24]:
processed_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,business_name,categories,categories_list,category,sentences,prompt_aspect,prompt_sentiment,prompt_aspect_lemm
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,I love driving on the Benjamin Franklin Bridge.,driving,positive,drive
1,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,"Every time I go up to NJ, I make sure I take this bridge.",bridge,neutral,bridge
2,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,The view from the bridge is amazing.,view,positive,view
3,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,I love the waterfront PA/NJ scenery.,scenery,positive,scenery
4,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,"I love driving on the Benjamin Franklin Bridge. Every time I go up to NJ, I make sure I take this bridge. The view from the bridge is amazing. I love the waterfront PA/NJ scenery. It's best to have EZ Pass during busy times. The EZ pass makes getting through the tolls a lot smoother. If you are riding across the Benjamin Franklin Bridge, enjoy the waterfront view.",1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government","[Automotive, Local Flavor, Active Life, Landmarks & Historical Buildings, Public Services & Government]",Automotive,It's

In [25]:
processed_df.to_pickle(f"../../data/yelp/reviews_absa_processed.pkl")